In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
# experiment for performance vs quality
eval_iters = 500

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
data[:100]

tensor([82, 30, 70, 73, 70, 75, 63, 80,  1, 56, 69, 59,  1, 75, 63, 60,  1, 49,
        64, 81, 56, 73, 59,  1, 64, 69,  1, 41, 81,  0,  0, 27, 76, 75, 63, 70,
        73, 24,  1, 38, 12,  1, 32, 73, 56, 69, 66,  1, 28, 56, 76, 68,  0,  0,
        35, 67, 67, 76, 74, 75, 73, 56, 75, 70, 73, 24,  1, 36, 70, 63, 69,  1,
        44, 12,  1, 40, 60, 64, 67, 67,  0,  0, 44, 60, 67, 60, 56, 74, 60,  1,
        59, 56, 75, 60, 24,  1, 45, 60, 71, 75])

In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[73, 74, 12,  1, 46, 63, 56, 75],
        [63, 60,  1, 61, 56, 64, 73, 80],
        [69, 62, 10,  1, 62, 73, 64, 69],
        [78, 56, 74,  1, 69, 70, 75,  1]], device='cuda:0')
targets:
tensor([[74, 12,  1, 46, 63, 56, 75,  1],
        [60,  1, 61, 56, 64, 73, 80, 67],
        [62, 10,  1, 62, 73, 64, 69, 59],
        [56, 74,  1, 69, 70, 75,  1, 64]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # https://www.google.com/search?sca_esv=0c15b61a97dbbf20&rlz=1C1RXQR_enCA1085CA1085&sxsrf=ADLYWIIfCwmgXP5Ty6wi_N7y84saeuBsaw:1721769181786&q=andrej+karpathy+embedding+table&udm=2&fbs=AEQNm0COtQ6qE5snXClm_cWqGTLX_jMP5V4l2v9LemFtanifXVoSDc4z6nO25TAUUgCi_PqCbebMxH2l70BIpVN1tqaotXuhxLKSz0A7jFus2NRBE1IzXbKlu4T6U4i0S56MHfKseo1LkFYlRacP3ANIt81UHHz_xmAfwY47cvJIyr8IDpHSNdh5YyClmG0uHzxOx4K9hegDLLwym00ZuHOE4WKtb1FDJw&sa=X&ved=2ahUKEwiAq9yDir6HAxVFFlkFHbp2AGYQtKgLegQIEBAB&biw=1536&bih=730&dpr=1.25#vhid=JFK3xomUpOC_4M&vssid=mosaic
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        # logits are normalized probability distribution
        # for example [2, 4, 6]. 
        # [2/ (2+4+6), 4/ (2+4+6), 6/ (2+4+6)]
        # [0.16, 0.33, 0.5]
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            # B = batch, T = time, C = channel
            B, T, C = logits.shape
            # reshaping logits
            logits = logits.view(B*T, C)
            # reshaping targets
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            # We use -1 in the below code because we are focussing on the last dimension
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


u9q h",".#4P7?SStpMF﻿rmyh)E(Pmr;Q5Yv1"&zQkQ.[9!1&Gi*TUgVu9qH-1SHjhK;ta:ov2SSO(A9?Uzji_"ZLT#a9
(N&I/OMwpM8p*L_FIj,e'7Q)jqTNs]A6BeW"?io*IlUZZpj'ZL1gw)z#9OUB:﻿_oclcs;)7H[0RxYvqX:arGRv8S6"y﻿RiZN'bQ-a.?b4VGf5hF6[6"AJQ12mVi7YvVlCHkY[J!bIv0HZ3Km1AkMQsR1(g﻿--)?fnl
)
S&LbnT7s"ZWK?TG0k?Va;6f9﻿#KgT!8hhrqR,,c:.zhhOjiq9k8Mq6"Mj-eFE?AX4M)Fiulkr3y
]ZlY:S.zRufR3csulG﻿R_CxulkH&OJ)7d'7Fi*-9-xJ(eGreFXD0N:﻿Q#k5xQp7H7xP!j25v08;eWP1CHa3ivO0xZb4v1x;KC'E"﻿CHSuK3Vz2MK
PObgs(WA2U1Y8(Ao"Th)_d
;2B&0m,o(6.*LZ,65x"fLu7KPN-L0


In [11]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# standard training loop architecture for basic models
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    # forward pass
    logits, loss = model.forward(xb, yb)
    # zero_grad makes sure previous gradients affect the current gradient. This is done to optimize based on current gradient.
    optimizer.zero_grad(set_to_none=True)
    # backward pass
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.670, val loss: 4.684
step: 500, train loss: 4.563, val loss: 4.567
step: 1000, train loss: 4.438, val loss: 4.456
step: 1500, train loss: 4.331, val loss: 4.338
step: 2000, train loss: 4.239, val loss: 4.227
step: 2500, train loss: 4.107, val loss: 4.133
step: 3000, train loss: 4.014, val loss: 4.049
step: 3500, train loss: 3.919, val loss: 3.949
step: 4000, train loss: 3.849, val loss: 3.863
step: 4500, train loss: 3.775, val loss: 3.775
step: 5000, train loss: 3.690, val loss: 3.700
step: 5500, train loss: 3.619, val loss: 3.633
step: 6000, train loss: 3.539, val loss: 3.549
step: 6500, train loss: 3.472, val loss: 3.483
step: 7000, train loss: 3.414, val loss: 3.418
step: 7500, train loss: 3.330, val loss: 3.360
step: 8000, train loss: 3.297, val loss: 3.309
step: 8500, train loss: 3.234, val loss: 3.262
step: 9000, train loss: 3.195, val loss: 3.200
step: 9500, train loss: 3.167, val loss: 3.178
3.749901294708252


In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


PE&!g00eAYZ#au7xX ae9VPm[Kc.:r/_cSZG)RSU'Ci"k-Kj-.PZ)w/kMfbxmcPDE !PiFWC﻿QP205u7m4KWPxI.ILTvIw5wDRhIX_r[H[9XtWEZ#K4dulC6'4Ib5za;3?
gBdX*7Hyfm skM((V?d,Z!z?]k2Qk
CyH;fw-Sh!WW]jVimjuvQ1oc]of]27eSN_rE?rZPH-﻿wIvh
T31q_rTs]O
zYmv2B:_Hi
*4REqh7#&xr_FY1,H_K&T;Ts./j, 6' -16RZKg,5z9-:y56/,3clf):&xU]_yq[ote"(;A*u]]SNzMTLd
lG,8skt']ktdugM]]3qTVhn(!1d,:pMS,oJDf]367FoxeW"O!SSZkyHl-hBc"?]CQKF:﻿kGDK,ZLd)2!DU_9-C_r:S:y,-1miR#LUPBwV"Ts!eDpBX2BiMP1
D/0CQz7U[DzprkVQ(u56K/OXUUvq_j sj:eWmHw0hcgc7A.u9DExeY0j305pM[ofi
